In [ ]:
import json
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
import re
import shutil
import time

load_dotenv(override=True)

In [ ]:
def extract_latest_screenshot_path(result):
    """
    Extract the latest (most recent) screenshot file path from an MCP result object.

    Supports:
    - structuredContent["path"]
    - meta["path"]
    - Any path inside text content (e.g. "/tmp/playwright-mcp-output/...png")

    Returns:
        str: The most recent screenshot path found.
    Raises:
        ValueError if no valid path is found.
    """
    paths = []

    # 1️⃣ structuredContent
    if hasattr(result, "structuredContent") and isinstance(result.structuredContent, dict):
        path = result.structuredContent.get("path")
        if path:
            paths.append(path)

    # 2️⃣ meta
    if hasattr(result, "meta") and isinstance(result.meta, dict):
        path = result.meta.get("path")
        if path:
            paths.append(path)

    # 3️⃣ text-based path detection in content
    if hasattr(result, "content") and result.content:
        for item in result.content:
            if hasattr(item, "text") and item.text:
                # Find all paths that look like /tmp/playwright-mcp-output/...png
                found = re.findall(r"/tmp/playwright-[\w/-]+\.png", item.text)
                paths.extend(found)

    # 4️⃣ Clean up duplicates
    paths = list(dict.fromkeys(paths))

    if not paths:
        raise ValueError(f"Could not determine screenshot path from result: {result}")

    # 5️⃣ Return the latest (last one in the list)
    latest_path = paths[-1]
    print(f"path : {latest_path}")
    return latest_path

In [ ]:
async def take_and_save_screenshot(session, label: str):
    """
    Take a screenshot via Playwright MCP and save it to a custom folder with a given label.
    Returns the final saved path.
    """
    await asyncio.sleep(0.5)

    # Call the MCP tool
    print("call mcp tool")
    result = await session.call_tool("browser_take_screenshot", {})

    # Try to find the actual temp screenshot path
    print("Construct path")
    tmp_path = extract_latest_screenshot_path(result)
    if not os.path.exists(tmp_path):
        raise FileNotFoundError(f"Screenshot file not found: {tmp_path}")

    # Prepare custom folder
    print("Prepare customer folder")
    base_dir = os.path.join(os.getcwd(), "screenshots")
    os.makedirs(base_dir, exist_ok=True)

    # Create a unique name
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    filename = f"{label}_{timestamp}.png"
    dest_path = os.path.join(base_dir, filename)

    # Copy file from MCP’s temp folder
    print("Ready to copy")
    shutil.copy(tmp_path, dest_path)
    print(f"📸 Saved screenshot: {dest_path}")

    return dest_path

In [ ]:
# Test Playwright MCP Server
async def test_playwright_server():
    
    # Connect to Playwright MCP server
    server_params = StdioServerParameters(
        command="npx",
        args=["@playwright/mcp@latest", "--browser", "chromium"],
        env={"DISPLAY": os.environ.get("DISPLAY", ":0")}
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # List available tools
            print("📋 Available Playwright Tools:")
            tools_response = await session.list_tools()
            for tool in tools_response.tools:
                print(f"  - {tool.name}")
            
            print("\n🎭 Testing Playwright Automation:")
            
            # Test 1: Navigate to a webpage
            print("\n1. Navigating to example.com...")
            result1 = await session.call_tool("browser_navigate", {
                "url": "https://example.com"
            })
            print(f"   Result: {result1.content[0].text[:200] if result1.content else 'Success'}")
            
            # Test 2: Take a screenshot
            print("Before taking screenshot1")
            
            dest_path = await take_and_save_screenshot(session, "test_screenshot.png")
            print(f" Screenshot saved at: {dest_path} ")
            #screenshot_path = os.path.join(screenshot_dir, "test_screenshot.png")
            #print(f"\n2. Taking screenshot to: {screenshot_path}")
            #result2 = await session.call_tool("browser_take_screenshot", {
            #    "path": screenshot_path
            #})
            #print(f"   Result: {result2.content[0].text if result2.content else 'Success'}")
            
            # Verify file exists
            #if os.path.exists(screenshot_path):
            #    print(f"   ✅ Screenshot saved! Size: {os.path.getsize(screenshot_path)} bytes")
            #else:
            #    print(f"   ❌ Screenshot not found at: {screenshot_path}")
            
            # Test 3: Navigate to todo app
            print("\n3. Navigating to todo app...")
            result3 = await session.call_tool("browser_navigate", {
                "url": "https://eviltester.github.io/simpletodolist/todo.html"
            })
            print(f"   Loaded todo app {result3}")

            # Wait for the input to be visible (optional, but good practice)
            await session.call_tool("browser_wait_for", {
                "selector": ".new-todo",
                "state": "visible",
                "timeout": 5000
            })

            print("\n4. Typing into todo input...")
            result4 = await session.call_tool("browser_type", {
                "element": "Enter new todo text here",
                "ref": "e8",
                "text": "Test task from Jupyter"
            })
            print(f"   Input typed! {result4}")

            """
            # Step 2. Take a fresh DOM snapshot (important!)
            await session.call_tool("browser_snapshot", {})
            
            # Test 4: Fill a form field
            print("\n4. Filling todo input...")
            result4 = await session.call_tool("browser_fill_form", {
                "fields": [
                    {
                        "name": "newTodo",
                        "type": "textbox",
                        "ref": ".new-todo",  # or input.new-todo
                        "value": "Test task from Jupyter"
                    }
                ]
            })
            print(f"   Input filled! {result4}")
            """
            await take_and_save_screenshot(session, "test_todo_input_filled.png")
            
            # Test 5: Press Enter key
            print("\n5. Pressing Enter key...")
            result5 = await session.call_tool("browser_press_key", {
                "key": "Enter"
            })
            print(f"   Key pressed!")
            
            # Test 6: Take final screenshot
            dest_path = await take_and_save_screenshot(session, "test_todo_screenshot.png")
            print(f" Screenshot saved at: {dest_path} ")
            """
            final_screenshot = os.path.join(screenshot_dir, "test_todo_screenshot.png")
            print(f"\n6. Taking final screenshot to: {final_screenshot}")
            result6 = await session.call_tool("browser_take_screenshot", {
                "path": final_screenshot
            })
            
            if os.path.exists(final_screenshot):
                print(f"   ✅ Final screenshot saved! Size: {os.path.getsize(final_screenshot)} bytes")
            else:
                print(f"   ❌ Screenshot not found!")
            """
            
            # Test 7: Get browser snapshot (page info)
            print("\n7. Getting page snapshot...")
            result7 = await session.call_tool("browser_snapshot", {})
            print(f"   Snapshot: {result7.content[0].text[:300] if result7.content else 'No content'}...")
            
            print("\n✅ All Playwright tests completed!")
            #print(f"📂 Check screenshots in: {screenshot_dir}")

In [ ]:
await test_playwright_server()